# 🗂️ Export Transcripts to Obsidian
Convert CSV transcripts to Markdown notes with Media Extended timestamp links, embedded validation snippets, and a quantitative questionnaire.

Workflow:
1. Load transcripts (CSV) and optional validation_report.json.
2. Create Obsidian-friendly Markdown with timestamp links to audio.
3. Append a removable validation block (italic tag) after each transcript.
4. Append the quantitative questionnaire at the end of each note.

In [ ]:
# 1) Paths and inputs
from pathlib import Path
import yaml

from qualivault import find_workspace_root

# ============================================
# PROJECT CONFIGURATION
# ============================================
PROJECT_NAME = 'YOUR_PROJECT_NAME'  # <-- Change this to work on different projects

# Auto-detect workspace root and project path
notebook_path = Path.cwd()
workspace_root = find_workspace_root(notebook_path)
project_root = workspace_root / 'projects' / PROJECT_NAME

if not project_root.exists():
    raise FileNotFoundError(
        f"❌ Project not found: {project_root}\n   Available projects in {workspace_root / 'projects'}:"
    )

config_path = project_root / 'config.yml'
if not config_path.exists():
    raise FileNotFoundError(f"❌ Config not found: {config_path}")

print(f"🎯 Working on project: {PROJECT_NAME}")
print(f"📁 Project root:       {project_root}")
print(f"⚙️  Config file:        {config_path}")
print()

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

paths_cfg = config.get('paths', {})

# Directories
TRANSCRIPTS_DIR = Path(paths_cfg.get('output_base_folder', project_root)).expanduser().resolve()

# Obsidian vault setup
obsidian_vault = Path(paths_cfg.get('obsidian_vault_folder', project_root)).expanduser().resolve()
compact_audio_subfolder = paths_cfg.get('compact_audio_folder', 'compact_audio')

AUDIO_VAULT_DIR = obsidian_vault / compact_audio_subfolder
OUTPUT_DIR = obsidian_vault

# Validation reports
VALIDATION_REPORTS_DIR = project_root / 'validation_reports'
LEGACY_VALIDATION_REPORT_PATH = project_root / 'validation_report.json'

print(f'📂 Transcripts:        {TRANSCRIPTS_DIR}')
print(f'📚 Obsidian Vault:     {obsidian_vault}')
print(f'🎧 Audio subfolder:    "{compact_audio_subfolder}"')
print(f'🎧 Audio vault path:   {AUDIO_VAULT_DIR}')
print(f'   ↳ Exists: {AUDIO_VAULT_DIR.exists()}')
if AUDIO_VAULT_DIR.exists():
    audio_count = len(list(AUDIO_VAULT_DIR.glob("*")))
    print(f'   ↳ Files found: {audio_count}')
print(f'📝 Output MD files:    {OUTPUT_DIR}')
print(f'✅ Validation reports: {VALIDATION_REPORTS_DIR} (exists={VALIDATION_REPORTS_DIR.exists()})')
print(f'✅ Legacy report file: {LEGACY_VALIDATION_REPORT_PATH} (exists={LEGACY_VALIDATION_REPORT_PATH.exists()})')


📂 Transcripts:       D:\legendary2\transcribe
📚 Obsidian Vault:    D:\legendary2\obsidian_vault
🎧 Audio subfolder:   "compact_audio"
🎧 Audio vault path:  D:\legendary2\obsidian_vault\compact_audio
   ↳ Exists: True
   ↳ Files found: 88
📝 Output MD files:   D:\legendary2\obsidian_vault
✅ Validation report: True


In [6]:
# 2) Questionnaire + settings
TARGET_SPEAKER = 'SPEAKER_00'  # bolded in output

QUESTION_SECTIONS = {
    'Section Q2: Demographics': [
        'Q2_1_person_interviewed', 'Q2_2_birthyear', 'Q2_3_residents',
        'Q2_4_year_aqcuisition', 'Q2_5_income_outside_farm'
    ],
    'Section Q3: Farm Structure': [
        'Q3_1_farmsize', 'Q3_2_leased_area', 'Q3_3_leased_out_area',
        'Q3_4_ownership_of_other_farms', 'Q3_5_started_farming',
        'Q3_6_ownership_motivation', 'Q3_7_organic', 'Q3_8_local_farming_conditions'
    ],
    'Section Q4: Crops & Livestock': [
        'Q4_1_crop_types', 'Q4_2_crop_reason', 'Q4_3_livestock',
        'Q4_4_livestock_type_count', 'Q4_5_livestock_changes_past',
        'Q4_6_livestock_changes_future'
    ],
    'Section Q5: Land Use Changes': [
        'Q5_1_arable_to_grass', 'Q5_2_grass_to_arable', 'Q5_3_arable_to_nature',
        'Q5_4_nature_to_arable', 'Q5_5_grass_to_nature', 'Q5_6_nature_to_grass',
        'Q5_7_other_changes', 'Q5_8_crop_changes_past', 'Q5_9_birds',
        'Q5_10_insects', 'Q5_11_other_animals'
    ],
    'Section Q6: Decision Making': [
        'Q6_1_planning_crop_rotation', 'Q6_2_considerations_crops',
        'Q6_3_crop_changes_future', 'Q6_4_soil_variation_crops',
        'Q6_5_soil_impact_cropchoice', 'Q6_6_terrain_impact_cropchoice',
        'Q6_7_drainage_impact_cropchoice', 'Q6_8_changes_fieldplanning',
        'Q6_9_good_agricultural_year'
    ],
    'Section Q7: Legumes': [
        'Q7_1_1_Legumes_crop_rotation', 'Q7_1_2_Challenges_legumes',
        'Q7_1_3_pros_cons_legumes', 'Q7_1_4_perception_legumes',
        'Q7_1_5_Perception_changes_legumes', 'Q7_1_6_advantages_disadvantages_legumes',
        'Q7_1_7_barriers_legumes', 'Q7_1_8_benefits_legumes', 'Q7_1_9_cooperation_sales',
        'Q7_2_1_experience_legumes', 'Q7_2_2_perception_legumes',
        'Q7_2_3_pros_cons_legumes', 'Q7_2_4_advantages_disadvantages_legumes',
        'Q7_2_5_expected_challenges_legumes', 'Q7_2_6_incentives_legumes',
        'Q7_2_7_benefits_legumes'
    ],
    'Section Q8: Field Conditions': [
        'Q8_1_best_worst_fields', 'Q8_2_drainage_system', 'Q8_3_waterlogged_fields',
        'Q8_4_drought_fields', 'Q8_5_plantprotection_fields',
        'Q8_6_plantprotection_use_change', 'Q8_7_production_conditions_fields',
        'Q8_8_production_security_fields', 'Q8_9_yield_data'
    ],
    'Section Q9: Society & Admin': [
        'Q9_1_farmtype', 'Q9_2_admin_factors_cropchoice', 'Q9_3_env_impact_choice',
        'Q9_4_societal_expectations', 'Q9_5_changes_societal_expectations',
        'Q9_6_traditions_crops', 'Q9_7_knowlegde_info_sources'
    ],
    'Section Q10: Landscape': [
        'Q10_1_landscape_role', 'Q10_2_landscape_role_changes', 'Q10_3_landscape_steward'
    ],
    'Section Q11: Future': [
        'Q11_1_future_farm_wishes', 'Q11_2_future_legumes', 'Q11_3_biodiversity_considerations',
        'Q11_4_climate_adaptation', 'Q11_5_nitrogen_management', 'Q11_6_retrospective_farm',
        'Q11_7_Green_tripartite'
    ]
}


In [ ]:
# 3) Workflow helpers (moved into module)
from qualivault import debug_audio_matching, export_transcripts_to_obsidian


In [ ]:
# 4) Debug (optional): audio file matching
# Set to False to skip debug output
RUN_AUDIO_MATCHING_DEBUG = False

if RUN_AUDIO_MATCHING_DEBUG:
    debug_audio_matching(
        transcripts_dir=TRANSCRIPTS_DIR,
        audio_vault_dir=AUDIO_VAULT_DIR,
        sample_files=3,
    )
else:
    print("(Skipping audio matching debug)")



🔍 DETAILED AUDIO FILE MATCHING DEBUG

Audio vault: D:\legendary2\obsidian_vault\compact_audio
Exists: True

Audio files found (88):
  • interview_1.mp3 (stem: interview_1)
  • interview_10.mp3 (stem: interview_10)
  • interview_102.mp3 (stem: interview_102)
  • interview_104.mp3 (stem: interview_104)
  • interview_108.mp3 (stem: interview_108)
  • interview_110.mp3 (stem: interview_110)
  • interview_111.mp3 (stem: interview_111)
  • interview_112.mp3 (stem: interview_112)
  • interview_114.mp3 (stem: interview_114)
  • interview_12.mp3 (stem: interview_12)
  • interview_120.mp3 (stem: interview_120)
  • interview_121.mp3 (stem: interview_121)
  • interview_122.mp3 (stem: interview_122)
  • interview_126.mp3 (stem: interview_126)
  • interview_128.mp3 (stem: interview_128)
  • interview_132.mp3 (stem: interview_132)
  • interview_135.mp3 (stem: interview_135)
  • interview_136.mp3 (stem: interview_136)
  • interview_137.mp3 (stem: interview_137)
  • interview_138.mp3 (stem: interview_

In [ ]:
# 5) Export CSV transcripts to Obsidian Markdown
# Resume behavior:
# - If `FORCE_OVERWRITE=False`, existing `.md` files are skipped
FORCE_OVERWRITE = False

exported, skipped = export_transcripts_to_obsidian(
    transcripts_dir=TRANSCRIPTS_DIR,
    output_dir=OUTPUT_DIR,
    obsidian_vault=obsidian_vault,
    compact_audio_subfolder=compact_audio_subfolder,
    target_speaker=TARGET_SPEAKER,
    question_sections=QUESTION_SECTIONS,
    validation_reports_dir=VALIDATION_REPORTS_DIR,
    legacy_validation_report_path=LEGACY_VALIDATION_REPORT_PATH,
    force=FORCE_OVERWRITE,
)



📝 EXPORTING 88 TRANSCRIPTS

🎧 ✅(7) Interview_1.md
🎧 ✅(11) Interview_10.md
🎧 ✅(16) Interview_102.md
🎧    Interview_104.md
🎧    Interview_108.md
🎧    Interview_110.md
🎧    Interview_111.md
🎧    Interview_112.md
🎧    Interview_114.md
🎧    Interview_12.md
🎧    Interview_120.md
🎧    Interview_121.md
🎧    Interview_122.md
🎧    Interview_126.md
🎧    Interview_128.md
🎧    Interview_132.md
🎧    Interview_135.md
🎧    Interview_136.md
🎧    Interview_137.md
🎧    Interview_138.md
🎧    Interview_139.md
🎧    Interview_14.md
🎧    Interview_140.md
🎧    Interview_145.md
🎧    Interview_150.md
🎧    Interview_151.md
🎧    Interview_152.md
🎧    Interview_154.md
🎧    Interview_156.md
🎧    Interview_158.md
🎧    Interview_16.md
🎧    Interview_160.md
🎧    Interview_166.md
🎧    Interview_175.md
🎧    Interview_177.md
🎧    Interview_178.md
🎧    Interview_179.md
🎧    Interview_180.md
🎧    Interview_181.md
🎧    Interview_182.md
🎧    Interview_183.md
🎧    Interview_184.md
🎧    Interview_187.md
🎧    Interview_194.md
🎧